<a href="https://colab.research.google.com/github/sunshineluyao/waiting-time-eip1559/blob/main/code/Data_Process/Process_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I: Set-up

In [1]:
# mount google drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# print the current directory
!pwd

/content


In [3]:
#@title path
#create a project folder and identify the path
path = "/content/drive/MyDrive/waitingtime" #@param {type:"string"}


In [5]:
#go to your project folder 
%cd /content/drive/MyDrive/waitingtime

/content/drive/MyDrive/waitingtime


In [6]:
# print the current directory again to make sure that you are in the project folder
!pwd 

/content/drive/MyDrive/waitingtime


In [7]:
import pandas as pd
import numpy as np

# Part II Import and Process the Waiting Time (Delay) Data

In [8]:
wt_merge =pd.read_pickle('wt_merge.pkl')
wt_merge  

,delay,included_in_block_num
0,6.961628,15543911
1,6.239315,15543911
2,13.702400,15543911
3,11.898386,15543911
4,12.329767,15543911
...,...,...
23006690,9.533371,15543911
23006691,9.454926,15543911
23006692,9.086987,15543911
23006693,8.778224,15543911


In [19]:
wt_merge=wt_merge.sort_values(by='included_in_block_num', ascending=True)
wt_merge.head()

,delay,included_in_block_num
12813299,4.297338,15467393
12813381,320.244233,15467393
12813380,18.564331,15467393
12813379,326.718764,15467393
12813378,331.905813,15467393


In [23]:
wt_merge.dtypes

delay                    float64
included_in_block_num      int64
dtype: object

In [29]:
# Define a lambda function
q_25 = lambda x: x.quantile(.25)
q_25.__name__ = 'q25%'
# Define a lambda function
q_75 = lambda x: x.quantile(.75)
q_75.__name__ = 'q75%'

In [30]:
# https://pbpython.com/groupby-agg.html
wt_agg_merge = wt_merge.groupby(['included_in_block_num'],as_index=False).agg({'delay': ['mean', q_25,q_75]})

wt_agg_merge.columns = [
'_'.join(col).rstrip('_') for col in wt_agg_merge.columns.values
]
wt_agg_merge

,included_in_block_num,delay_mean,delay_q25%,delay_q75%
0,15467393,139.474531,3.128987,328.601833
1,15467394,58.586977,3.918423,5.125737
2,15467395,216.659667,4.111789,413.183778
3,15467396,300.055710,255.420531,394.048948
4,15467397,3.448906,-3.511802,7.359402
...,...,...,...,...
134950,15607389,11.960169,11.162012,12.424321
134951,15607390,16.537019,10.048979,20.423403
134952,15607391,10.040779,6.404940,14.110622
134953,15607392,14.851740,8.140298,11.451282


In [31]:
wt_agg_merge['IQR'] =wt_agg_merge['delay_q75%']-wt_agg_merge['delay_q25%']
wt_agg_merge

,included_in_block_num,delay_mean,delay_q25%,delay_q75%,IQR
0,15467393,139.474531,3.128987,328.601833,325.472846
1,15467394,58.586977,3.918423,5.125737,1.207314
2,15467395,216.659667,4.111789,413.183778,409.071989
3,15467396,300.055710,255.420531,394.048948,138.628417
4,15467397,3.448906,-3.511802,7.359402,10.871204
...,...,...,...,...,...
134950,15607389,11.960169,11.162012,12.424321,1.262308
134951,15607390,16.537019,10.048979,20.423403,10.374424
134952,15607391,10.040779,6.404940,14.110622,7.705682
134953,15607392,14.851740,8.140298,11.451282,3.310984


# Part III: Import and Process the Ethereum Blockchain data

In [16]:
blocks_merge =pd.read_pickle('merge_blocks_selected.pkl')
blocks_merge

,number,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas
0,15467493,35021,30000000,7599434,1662237222,52,9787352044
1,15467593,47208,29999972,12111413,1662238920,156,9563105534
2,15467494,6705,30000000,1674394,1662237229,20,9183752505
3,15467793,139953,30029295,30010865,1662241702,945,2970262901
4,15467594,65678,30000000,15036069,1662238935,151,9332907583
...,...,...,...,...,...,...,...
139996,15607391,64874,30000000,11774830,1664072807,128,4745402708
139997,15607390,65869,30000000,16141399,1664072795,187,4700691339
139998,15607392,53133,30000000,11925182,1664072819,83,4617863288
139999,15607393,28186,30000000,4127909,1664072831,42,4499537546


In [20]:
blocks_merge=blocks_merge.sort_values(by='number', ascending=True)
blocks_merge

,number,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas
12,15467393,48542,30058619,9386622,1662235839,124,15206420881
16,15467394,2885,30029266,3551534,1662235846,9,14492773023
20,15467395,40646,30000000,6883047,1662235850,86,13109688169
25,15467396,75760,30000000,16728892,1662235852,171,12222932147
30,15467397,22503,30000000,3829781,1662235854,41,12399033227
...,...,...,...,...,...,...,...
139994,15607389,6064,30000000,1054445,1664072783,19,5318805455
139997,15607390,65869,30000000,16141399,1664072795,187,4700691339
139996,15607391,64874,30000000,11774830,1664072807,128,4745402708
139998,15607392,53133,30000000,11925182,1664072819,83,4617863288


# Part IV: Merge the waiting time (delay) data and the blockchain data

In [33]:
merge = blocks_merge.merge(wt_agg_merge, left_on='number', right_on='included_in_block_num', suffixes=(False, False))
merge

,number,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,included_in_block_num,delay_mean,delay_q25%,delay_q75%,IQR
0,15467393,48542,30058619,9386622,1662235839,124,15206420881,15467393,139.474531,3.128987,328.601833,325.472846
1,15467394,2885,30029266,3551534,1662235846,9,14492773023,15467394,58.586977,3.918423,5.125737,1.207314
2,15467395,40646,30000000,6883047,1662235850,86,13109688169,15467395,216.659667,4.111789,413.183778,409.071989
3,15467396,75760,30000000,16728892,1662235852,171,12222932147,15467396,300.055710,255.420531,394.048948,138.628417
4,15467397,22503,30000000,3829781,1662235854,41,12399033227,15467397,3.448906,-3.511802,7.359402,10.871204
...,...,...,...,...,...,...,...,...,...,...,...,...
134950,15607389,6064,30000000,1054445,1664072783,19,5318805455,15607389,11.960169,11.162012,12.424321,1.262308
134951,15607390,65869,30000000,16141399,1664072795,187,4700691339,15607390,16.537019,10.048979,20.423403,10.374424
134952,15607391,64874,30000000,11774830,1664072807,128,4745402708,15607391,10.040779,6.404940,14.110622,7.705682
134953,15607392,53133,30000000,11925182,1664072819,83,4617863288,15607392,14.851740,8.140298,11.451282,3.310984


In [34]:
merge =merge.drop(columns=['included_in_block_num'])
merge

,number,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,delay_mean,delay_q25%,delay_q75%,IQR
0,15467393,48542,30058619,9386622,1662235839,124,15206420881,139.474531,3.128987,328.601833,325.472846
1,15467394,2885,30029266,3551534,1662235846,9,14492773023,58.586977,3.918423,5.125737,1.207314
2,15467395,40646,30000000,6883047,1662235850,86,13109688169,216.659667,4.111789,413.183778,409.071989
3,15467396,75760,30000000,16728892,1662235852,171,12222932147,300.055710,255.420531,394.048948,138.628417
4,15467397,22503,30000000,3829781,1662235854,41,12399033227,3.448906,-3.511802,7.359402,10.871204
...,...,...,...,...,...,...,...,...,...,...,...
134950,15607389,6064,30000000,1054445,1664072783,19,5318805455,11.960169,11.162012,12.424321,1.262308
134951,15607390,65869,30000000,16141399,1664072795,187,4700691339,16.537019,10.048979,20.423403,10.374424
134952,15607391,64874,30000000,11774830,1664072807,128,4745402708,10.040779,6.404940,14.110622,7.705682
134953,15607392,53133,30000000,11925182,1664072819,83,4617863288,14.851740,8.140298,11.451282,3.310984


In [35]:
merge['ETH2.0'] = 0
merge.loc[merge.number>=15537393, 'ETH2.0']=1
merge

,number,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,delay_mean,delay_q25%,delay_q75%,IQR,ETH2.0
0,15467393,48542,30058619,9386622,1662235839,124,15206420881,139.474531,3.128987,328.601833,325.472846,0
1,15467394,2885,30029266,3551534,1662235846,9,14492773023,58.586977,3.918423,5.125737,1.207314,0
2,15467395,40646,30000000,6883047,1662235850,86,13109688169,216.659667,4.111789,413.183778,409.071989,0
3,15467396,75760,30000000,16728892,1662235852,171,12222932147,300.055710,255.420531,394.048948,138.628417,0
4,15467397,22503,30000000,3829781,1662235854,41,12399033227,3.448906,-3.511802,7.359402,10.871204,0
...,...,...,...,...,...,...,...,...,...,...,...,...
134950,15607389,6064,30000000,1054445,1664072783,19,5318805455,11.960169,11.162012,12.424321,1.262308,1
134951,15607390,65869,30000000,16141399,1664072795,187,4700691339,16.537019,10.048979,20.423403,10.374424,1
134952,15607391,64874,30000000,11774830,1664072807,128,4745402708,10.040779,6.404940,14.110622,7.705682,1
134953,15607392,53133,30000000,11925182,1664072819,83,4617863288,14.851740,8.140298,11.451282,3.310984,1


In [36]:
merge['merge_bn'] = merge['number']-15537393
merge

,number,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,delay_mean,delay_q25%,delay_q75%,IQR,ETH2.0,merge_bn
0,15467393,48542,30058619,9386622,1662235839,124,15206420881,139.474531,3.128987,328.601833,325.472846,0,-70000
1,15467394,2885,30029266,3551534,1662235846,9,14492773023,58.586977,3.918423,5.125737,1.207314,0,-69999
2,15467395,40646,30000000,6883047,1662235850,86,13109688169,216.659667,4.111789,413.183778,409.071989,0,-69998
3,15467396,75760,30000000,16728892,1662235852,171,12222932147,300.055710,255.420531,394.048948,138.628417,0,-69997
4,15467397,22503,30000000,3829781,1662235854,41,12399033227,3.448906,-3.511802,7.359402,10.871204,0,-69996
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134950,15607389,6064,30000000,1054445,1664072783,19,5318805455,11.960169,11.162012,12.424321,1.262308,1,69996
134951,15607390,65869,30000000,16141399,1664072795,187,4700691339,16.537019,10.048979,20.423403,10.374424,1,69997
134952,15607391,64874,30000000,11774830,1664072807,128,4745402708,10.040779,6.404940,14.110622,7.705682,1,69998
134953,15607392,53133,30000000,11925182,1664072819,83,4617863288,14.851740,8.140298,11.451282,3.310984,1,69999


In [37]:
merge['gas_used_pct'] = merge['gas_used']/merge['gas_limit']
merge

,number,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,delay_mean,delay_q25%,delay_q75%,IQR,ETH2.0,merge_bn,gas_used_pct
0,15467393,48542,30058619,9386622,1662235839,124,15206420881,139.474531,3.128987,328.601833,325.472846,0,-70000,0.312277
1,15467394,2885,30029266,3551534,1662235846,9,14492773023,58.586977,3.918423,5.125737,1.207314,0,-69999,0.118269
2,15467395,40646,30000000,6883047,1662235850,86,13109688169,216.659667,4.111789,413.183778,409.071989,0,-69998,0.229435
3,15467396,75760,30000000,16728892,1662235852,171,12222932147,300.055710,255.420531,394.048948,138.628417,0,-69997,0.557630
4,15467397,22503,30000000,3829781,1662235854,41,12399033227,3.448906,-3.511802,7.359402,10.871204,0,-69996,0.127659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134950,15607389,6064,30000000,1054445,1664072783,19,5318805455,11.960169,11.162012,12.424321,1.262308,1,69996,0.035148
134951,15607390,65869,30000000,16141399,1664072795,187,4700691339,16.537019,10.048979,20.423403,10.374424,1,69997,0.538047
134952,15607391,64874,30000000,11774830,1664072807,128,4745402708,10.040779,6.404940,14.110622,7.705682,1,69998,0.392494
134953,15607392,53133,30000000,11925182,1664072819,83,4617863288,14.851740,8.140298,11.451282,3.310984,1,69999,0.397506


In [38]:
merge['timestamp'] = pd.to_datetime(merge['timestamp'], unit='s')
merge

,number,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,delay_mean,delay_q25%,delay_q75%,IQR,ETH2.0,merge_bn,gas_used_pct
0,15467393,48542,30058619,9386622,2022-09-03 20:10:39,124,15206420881,139.474531,3.128987,328.601833,325.472846,0,-70000,0.312277
1,15467394,2885,30029266,3551534,2022-09-03 20:10:46,9,14492773023,58.586977,3.918423,5.125737,1.207314,0,-69999,0.118269
2,15467395,40646,30000000,6883047,2022-09-03 20:10:50,86,13109688169,216.659667,4.111789,413.183778,409.071989,0,-69998,0.229435
3,15467396,75760,30000000,16728892,2022-09-03 20:10:52,171,12222932147,300.055710,255.420531,394.048948,138.628417,0,-69997,0.557630
4,15467397,22503,30000000,3829781,2022-09-03 20:10:54,41,12399033227,3.448906,-3.511802,7.359402,10.871204,0,-69996,0.127659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134950,15607389,6064,30000000,1054445,2022-09-25 02:26:23,19,5318805455,11.960169,11.162012,12.424321,1.262308,1,69996,0.035148
134951,15607390,65869,30000000,16141399,2022-09-25 02:26:35,187,4700691339,16.537019,10.048979,20.423403,10.374424,1,69997,0.538047
134952,15607391,64874,30000000,11774830,2022-09-25 02:26:47,128,4745402708,10.040779,6.404940,14.110622,7.705682,1,69998,0.392494
134953,15607392,53133,30000000,11925182,2022-09-25 02:26:59,83,4617863288,14.851740,8.140298,11.451282,3.310984,1,69999,0.397506


In [42]:
merge['date'] =merge['timestamp'].dt.date
merge

,number,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,delay_mean,delay_q25%,delay_q75%,IQR,ETH2.0,merge_bn,gas_used_pct,date
0,15467393,48542,30058619,9386622,2022-09-03 20:10:39,124,15206420881,139.474531,3.128987,328.601833,325.472846,0,-70000,0.312277,2022-09-03
1,15467394,2885,30029266,3551534,2022-09-03 20:10:46,9,14492773023,58.586977,3.918423,5.125737,1.207314,0,-69999,0.118269,2022-09-03
2,15467395,40646,30000000,6883047,2022-09-03 20:10:50,86,13109688169,216.659667,4.111789,413.183778,409.071989,0,-69998,0.229435,2022-09-03
3,15467396,75760,30000000,16728892,2022-09-03 20:10:52,171,12222932147,300.055710,255.420531,394.048948,138.628417,0,-69997,0.557630,2022-09-03
4,15467397,22503,30000000,3829781,2022-09-03 20:10:54,41,12399033227,3.448906,-3.511802,7.359402,10.871204,0,-69996,0.127659,2022-09-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134950,15607389,6064,30000000,1054445,2022-09-25 02:26:23,19,5318805455,11.960169,11.162012,12.424321,1.262308,1,69996,0.035148,2022-09-25
134951,15607390,65869,30000000,16141399,2022-09-25 02:26:35,187,4700691339,16.537019,10.048979,20.423403,10.374424,1,69997,0.538047,2022-09-25
134952,15607391,64874,30000000,11774830,2022-09-25 02:26:47,128,4745402708,10.040779,6.404940,14.110622,7.705682,1,69998,0.392494,2022-09-25
134953,15607392,53133,30000000,11925182,2022-09-25 02:26:59,83,4617863288,14.851740,8.140298,11.451282,3.310984,1,69999,0.397506,2022-09-25


In [43]:
merge.to_csv('merge.csv')
merge.to_pickle('merge.pkl')